# 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

cloud jupyter 를 열고 aiffel/recommendata_iu/ 디렉토리에 python3로 ipynb파일을 생성 해 주세요 jupyter notebook 환경에서 프로젝트 코드를 작성하고 제출하시면 됩니다.

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결 해 줍니다.

Cloud shell에서 아래 명령어를 입력해주세요.

In [22]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/EXP_09_recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [23]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [24]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [25]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [26]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/EXP_09_recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.



### 2) 분석해 봅시다.

 - ratings에 있는 유니크한 영화 개수
 - ratings에 있는 유니크한 사용자 수
 - 가장 인기 있는 영화 30개(인기순)


In [31]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()


# ratings 와 movies 데이터 프레임 합치기
data = ratings.join(movies.set_index('movie_id'), on='movie_id')
data.head()



,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [32]:

# user_id가 1인 사용자가 어떤 영화를 시청했는지 확인해봅니다.
movie_id_user1 = data[data['user_id']== 1]['title']
movie_id_user1

0                One Flew Over the Cuckoo's Nest (1975)
1                      James and the Giant Peach (1996)
2                                   My Fair Lady (1964)
3                                Erin Brockovich (2000)
4                                  Bug's Life, A (1998)
5                            Princess Bride, The (1987)
6                                        Ben-Hur (1959)
7                             Christmas Story, A (1983)
8                Snow White and the Seven Dwarfs (1937)
9                              Wizard of Oz, The (1939)
10                          Beauty and the Beast (1991)
11                                          Gigi (1958)
12                        Miracle on 34th Street (1947)
13                      Ferris Bueller's Day Off (1986)
14                           Sound of Music, The (1965)
15                                     Airplane! (1980)
16                                        Tarzan (1999)
17                                         Bambi

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

### 4) CSR matrix를 직접 만들어 봅시다

In [ ]:

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.